In [27]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import math
import random
import community as luvain
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


In [2]:
period1 = pd.read_csv('dataset/Period1.csv')
period2 = pd.read_csv('dataset/Period2.csv')
test_data = pd.read_csv('dataset/TestData.csv')
period1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154836 entries, 0 to 154835
Data columns (total 3 columns):
year         154836 non-null int64
source id    154836 non-null int64
target id    154836 non-null int64
dtypes: int64(3)
memory usage: 3.5 MB


In [3]:
def find_unique_node(input_node_set1, input_node_set2):
    node_set = pd.concat([input_node_set1, input_node_set2], ignore_index=True)
    node_set_unique = node_set.loc[node_set.duplicated() == False]
    node_set_unique = node_set_unique.reset_index(drop=True)
    return node_set_unique


period1_node_unique = find_unique_node(period1['source id'], period1['target id'])
period2_node_unique = find_unique_node(period2['source id'], period2['target id'])
period_all_node_unique = find_unique_node(period1_node_unique, period2_node_unique)

# Format edge to newortkx type
period1_edge = [(period1.loc[i, 'source id'], period1.loc[i, 'target id']) for i in range(len(period1))]
period2_edge = [(period2.loc[i, 'source id'], period2.loc[i, 'target id']) for i in range(len(period2))]
period_all_edge = period1_edge + period2_edge


print('period1 ', 'node:', len(period1_node_unique.values), 'edge:', len(period1_edge))
print('period2 ', 'node:', len(period2_node_unique.values), 'edge:', len(period2_edge))
print('period_all ', 'node:', len(period_all_node_unique.values), 'edge:', len(period_all_edge))
print('period_test ', 'node:', len(period_all_node_unique.values), 'edge:', len(period_all_edge))

period1  node: 17028 edge: 154836
period2  node: 15565 edge: 98353
period_all  node: 23237 edge: 253189
period_test  node: 23237 edge: 253189


In [4]:
# Network generate (period1)
network_period1 = nx.Graph()
# Add nodes
network_period1.add_nodes_from(period_all_node_unique.values)
# Add edges
network_period1.add_edges_from(period1_edge)

print('node:', network_period1.number_of_nodes(), 'edge:', network_period1.number_of_edges())

node: 23237 edge: 154698


In [5]:
# network graph
sub_graph = network_period1.subgraph(list(period1_node_unique[0:100]))   # 原圖太大取前幾個邊出來畫
pos = nx.spring_layout(sub_graph)  # 圖的畫法
nx.draw(sub_graph, pos=pos, node_size=40, vim=0.0, vmax=1.0, node_color="red")

In [6]:
# Network generate (period_all)
network_all = nx.Graph()
# Add nodes
network_all.add_nodes_from(period_all_node_unique.values)
# Add edges
network_all.add_edges_from(period_all_edge)
print('node:', network_all.number_of_nodes(), 'edge:', network_all.number_of_edges())

node: 23237 edge: 252854


In [7]:
# common neighbor score (neighbor = 1 best)
def common_neighbor(network, input_node1, input_node2): 
    source_neighbor = [n for n in network.neighbors(input_node1)]
    target_neighbor = [n for n in network.neighbors(input_node2)]
    intersection = list(set(source_neighbor) & set(target_neighbor))
    return len(intersection)

# Jaccard's cofficient
def jaccard_cofficient(network, input_node1, input_node2):
    cofficient = 0
    source_neighbor = [n for n in network.neighbors(input_node1)]
    target_neighbor = [n for n in network.neighbors(input_node2)]
    union = list(set(source_neighbor) | set(target_neighbor))
    intersection = list(set(source_neighbor) & set(target_neighbor))
    if len(union) == 0:
        return 0
    else:
        return (len(intersection) / len(union))


# Adamic/Adar
def adamic_adar(network, input_node1, input_node2):
    adamic_score = 0
    source_neighbor = [n for n in network.neighbors(input_node1)]
    target_neighbor = [n for n in network.neighbors(input_node2)]
    intersection = list(set(source_neighbor) & set(target_neighbor))
    
    if len(intersection) == 0:
        return 0
    else:
        for v in intersection:
            adamic_score += 1 / math.log(len([nv for nv in network.neighbors(v)]))
        return adamic_score


# clustering coefficient
def clustering_coefficient(network, input_node):
    node_degree = network.degree[input_node]
    node_triangle = nx.triangles(network, input_node)
    if node_degree - 1 <= 0:
        return 0
    else:
        return (2 * node_triangle) / (node_degree * (node_degree - 1))   
    

# perferential attachment
def perferential_attachment(network, input_node1, input_node2): 
    source_neighbor = len([n for n in network.neighbors(input_node1)])
    target_neighbor = len([n for n in network.neighbors(input_node2)])
    return {'pa_mul': source_neighbor * target_neighbor, 'pa_add': source_neighbor + target_neighbor}

In [71]:
# Katz score
def katz_score(network, input_node1, input_node2, beta=0.005):
    score = 0
    for cutoff in range(1, 5):
        path_amount = 0
        all_paths = nx.all_simple_paths(network, input_node1, input_node2, cutoff=cutoff)
        for path in all_paths:
            if path_amount > 100:
                continue
            else:
                path_amount += 1
                
        score += (beta**cutoff) * path_amount
    return score

# katz_score = katz(network_test, test_data['source id'][709], test_data['target id'][709], beta=0.005)

In [47]:
# hits


In [62]:
# step 2 geneate model
# find period1 complement graph (使用補圖的所有edge 當training data才對，不過我電腦跑不動)
# network_period1_complement = nx.complement(network_period1)
# print("complement down")
# complement_edge = network_period1_complement.edges()

# 直接使用period2裡面的當1，再從network_period1 亂數取幾個點，他們形成的補圖edge當0
# period1 subgraph
period1_not_in_2 = period1_node_unique.append(period2_node_unique).drop_duplicates(keep=False)
period1_node_shuffle = random.Random(2).sample(list(period1_not_in_2), 450)
sub_graph = network_period1.subgraph(period1_node_shuffle)
sub_graph_complement = nx.complement(sub_graph)
# pos = nx.spring_layout(sub_graph)  # 圖的畫法
# nx.draw(sub_graph_complement, pos=pos, node_size=40, vim=0.0, vmax=1.0, node_color="red")

# tag label
train_label = []
# 注意順序
for edge in period2_edge:
    train_label.append(1)
for edge in list(sub_graph_complement.edges()):
    train_label.append(0)   

train_data_edge = period2_edge + list(sub_graph_complement.edges())
train_data = pd.DataFrame(data={'edge': train_data_edge, 'label': train_label})
train_data = shuffle(train_data, random_state=32).reset_index(drop=True)
train_data.head()

,edge,label
0,"(3273, 9901003)",1
1,"(10172, 9312198)",0
2,"(9707193, 9609123)",0
3,"(9061, 9805170)",1
4,"(102045, 9901128)",1


In [63]:
# train data feature
cn, jaccard, adamic, cc_mul, cc_add, pa_mul, pa_add = [],[], [], [], [], [], []
katz = []
source_id, target_id = [], []

netwrok_period2 = network_period1
netwrok_period2.add_nodes_from(period2_node_unique)

for edge_id, edge in enumerate(train_data['edge'].values):
    source_id.append(edge[0])
    target_id.append(edge[1])
    
    # neighbor base
    cn.append(common_neighbor(netwrok_period2, edge[0], edge[1]))
    jaccard.append(jaccard_cofficient(netwrok_period2, edge[0], edge[1]))
    adamic.append(adamic_adar(netwrok_period2, edge[0], edge[1]))
    
    # path base
    katz.append(katz_score(netwrok_period2, edge[0], edge[1]))
    # other
    source_cc = clustering_coefficient(netwrok_period2, edge[0])
    target_cc = clustering_coefficient(netwrok_period2, edge[1])
    cc_mul.append(source_cc * target_cc)
    cc_add.append(source_cc + target_cc)
    pa = perferential_attachment(netwrok_period2, edge[0], edge[1])
    pa_mul.append(pa['pa_mul'])
    pa_add.append(pa['pa_add'])
    
    if edge_id % 10000 == 0:
        print(edge_id, len(train_data))
# 
train_data['source id'] = pd.Series(source_id, index=train_data.index)
train_data['target id'] = pd.Series(target_id, index=train_data.index)
train_data['cn'] = pd.Series(cn, index=train_data.index)
train_data['jaccard'] = pd.Series(jaccard, index=train_data.index)
train_data['adamic'] = pd.Series(adamic, index=train_data.index)
train_data['cc_mul'] = pd.Series(cc_mul, index=train_data.index)
train_data['cc_add'] = pd.Series(cc_add, index=train_data.index)
train_data['pa_mul'] = pd.Series(pa_mul, index=train_data.index)
train_data['pa_add'] = pd.Series(pa_add, index=train_data.index)
train_data['katz'] = pd.Series(katz, index=train_data.index)
train_data.head(10)

0 199365


KeyboardInterrupt: 

In [12]:
#save train data
train_data_bye = train_data.drop('edge', axis=1)
train_data_bye.to_csv("train_data_real.csv", index=False)

In [73]:
#load train data
train_data = pd.read_csv("train_data_real.csv")

In [74]:
print('gogogo')
katz = []
for edge_id, edge in enumerate(train_data_edge):
    katz.append(katz_score(netwrok_period2, edge[0], edge[1]))
    if edge_id % 10000 == 0:
        print(edge_id, len(train_data))
        
train_data['katz'] = pd.Series(katz, index=train_data.index)
train_data.head(10)

gogogo
0 199365
10000 199365
20000 199365
30000 199365
40000 199365
50000 199365
60000 199365
70000 199365
80000 199365
90000 199365
100000 199365
110000 199365
120000 199365
130000 199365
140000 199365
150000 199365
160000 199365
170000 199365
180000 199365
190000 199365


,label,source id,target id,cn,jaccard,adamic,cc_mul,cc_add,pa_mul,pa_add,katz
0,1,3273,9901003,0,0.0,0.0,0.00,0.352632,0,20,0.0
1,0,10172,9312198,0,0.0,0.0,0.00,0.266667,0,10,0.0
2,0,9707193,9609123,0,0.0,0.0,0.00,0.351724,60,32,0.0
3,1,9061,9805170,0,0.0,0.0,0.00,0.198859,0,68,0.0
4,1,102045,9901128,0,0.0,0.0,0.00,0.148867,0,103,0.0
5,0,9410076,9705028,0,0.0,0.0,0.05,0.483333,48,19,0.0
6,1,111284,9808095,0,0.0,0.0,0.00,0.337662,0,22,0.0
7,1,8127,3220,0,0.0,0.0,0.00,0.000000,0,0,0.0
8,1,107149,9806106,0,0.0,0.0,0.00,0.148148,0,27,0.0
9,1,4186,9804123,0,0.0,0.0,0.00,0.211429,0,50,0.0


In [ ]:
train_data_bye.to_csv("train_data_katz.csv", index=False)

In [13]:
# train feature (pa_add 爛
feature = zip(train_data['cn'], train_data['jaccard'], train_data['adamic'], train_data['cc_mul'], train_data['cc_add'],
              train_data['pa_mul'])
feature = [[cn, jaccard, adamic, cc_mul, cc_add, pa_mul]
           for cn, jaccard, adamic, cc_mul, cc_add, pa_mul in feature]

In [ ]:
#SVM gogo
svm = SVC()
svm.fit(feature, train_data['label'].tolist())


In [ ]:
svm.predict([[0, 0, 0, 0, 0]])

In [11]:
# Decision tree
dt = tree.DecisionTreeClassifier()
dt = dt.fit(feature, train_data['label'].tolist())

In [12]:
# Random forest
rf = RandomForestClassifier(random_state=0,n_estimators=300)
rf = dt.fit(feature, train_data['label'].tolist())

In [13]:
# KNN
knn = KNeighborsClassifier(n_neighbors=3)
knn = knn.fit(feature, train_data['label'].tolist())

In [10]:
# step 3 link predict
# check test data is in network
test_data_node_unique = find_unique_node(test_data['source id'], test_data['target id'])
test_data_in_network = pd.concat([period_all_node_unique, test_data_node_unique], ignore_index=True)
test_data_in_network = test_data_in_network.loc[test_data_in_network.duplicated() == True]
test_data_in_network = test_data_in_network.reset_index(drop=True)


# Add test data node & edge in network
test_edge = [(test_data.loc[i, 'source id'], test_data.loc[i, 'target id']) for i in range(len(test_data))]
network_test = network_all
network_test.add_nodes_from(test_data_node_unique.values)
# network_test.add_edges_from(test_edge)

print('period_test ', 'node:', network_test.number_of_nodes(), 'edge:', network_test.number_of_edges())

period_test  node: 31491 edge: 252854


In [15]:
# test data feature 
neighbor_score_list = []
for data_id, data in test_data.iterrows():
    neighbor_score_list.append(common_neighbor(network_test, data['source id'], data['target id']))
    
print(neighbor_score_list[709])

5


In [16]:
jaccard_score_list = []
    
for data_id, data in test_data.iterrows():
    jaccard_score_list.append(jaccard_cofficient(network_test, data['source id'], data['target id']))
        
# Networkx jaccard_coefficient function
# networkx_jaccard = []
# fucking_list = []
# 
# for data_id, data in test_data.iterrows():
#     fucking_list.append(((data['source id'],data['target id'])))
#     
# preds = nx.jaccard_coefficient(network_all, fucking_list)
# for u,v,p in preds:
#     networkx_jaccard.append(p)
# 
# print(networkx_jaccard[709])
print(jaccard_score_list[709])

0.07692307692307693


In [17]:
adamic_score_list = []
for data_id, data in test_data.iterrows():
    adamic_score_list.append(adamic_adar(network_test, data['source id'], data['target id']))   


# Networkx adamic_adar_index
# networkx_adamic_adar_index = []
# fucking_list = []
# 
# for data_id, data in test_data.iterrows():
#     fucking_list.append(((data['source id'],data['target id'])))
# 
# preds = nx.adamic_adar_index(network_all, fucking_list)
# for u, v, p in preds:
#     networkx_adamic_adar_index.append(p)
#     
# print(networkx_adamic_adar_index[709])
print(adamic_score_list[709])

1.2312765119577433


In [18]:
cc_mul_score_list = []
cc_add_scroe_list = []
for data_id, data in test_data.iterrows():
    source_cc = clustering_coefficient(network_test, data['source id'])
    target_cc = clustering_coefficient(network_test, data['target id'])
    cc_mul_score_list.append(source_cc * target_cc) 
    cc_add_scroe_list.append(source_cc + target_cc)
    
# Networkx cc
# networkx_cc = []
# for data_id, data in test_data.iterrows():
#     cc_score = nx.clustering(network_test, data['source id']) * nx.clustering(network_test, data['target id'])
#     networkx_cc.append(cc_score)
# 
# print(networkx_cc[709])

print(cc_add_scroe_list[709])


1.0617702448210922


In [19]:
pa_mul_score_list = []
pa_add_scroe_list = []
for data_id, data in test_data.iterrows():
    pa = perferential_attachment(network_test, data['source id'], data['target id'])
    pa_mul_score_list.append(pa['pa_mul'])
    pa_add_scroe_list.append(pa['pa_add'])

print(pa_mul_score_list[709])

600


In [ ]:
latz_score_list = []
for data_id, data in test_data.iterrows():
    katz = katz_score(network_test, data['source id'], data['target id'])
    latz_score_list.append(katz)

In [23]:
# predict test data dt

test_feature = [[n, j, a, cc_mul, cc_add, pa_mul, katz] for n, j, a, cc_mul, cc_add, pa_mul, katz in 
                zip(neighbor_score_list, jaccard_score_list, adamic_score_list, cc_mul_score_list, cc_add_scroe_list,
                    pa_mul_score_list, latz_score_list)]

predict = knn.predict(test_feature)
print(len(predict))

10000


In [24]:
# output predict
row = [i for i in range(1, 10001)]
label = []


data = {'target id': row, 'label': label}
predict = pd.DataFrame(data=data, columns=['target id', 'label'])
predict.to_csv("predict/knn_nei_cc_real.csv", index=False)
